In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pii-detection-removal-from-educational-data/sample_submission.csv
/kaggle/input/pii-detection-removal-from-educational-data/train.json
/kaggle/input/pii-detection-removal-from-educational-data/test.json
/kaggle/input/data-exploration/deberta_test.csv
/kaggle/input/data-exploration/roberta_test.csv
/kaggle/input/data-exploration/roberta_train.csv
/kaggle/input/data-exploration/__results__.html
/kaggle/input/data-exploration/docs2cut.txt
/kaggle/input/data-exploration/deberta_train.csv
/kaggle/input/data-exploration/__notebook__.ipynb
/kaggle/input/data-exploration/__output__.json
/kaggle/input/data-exploration/train.csv
/kaggle/input/data-exploration/test.csv
/kaggle/input/data-exploration/custom.css


In [2]:
!pip install seqeval evaluate -q
import json
import argparse
from itertools import chain
from functools import partial
import torch
from transformers import AutoTokenizer,AutoModelForTokenClassification,Trainer, TrainingArguments, AutoConfig, DataCollatorForTokenClassification, TrainerCallback
import evaluate
from datasets import Dataset, features
import copy
from ast import literal_eval
import ast

2024-05-02 17:09:38.478832: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 17:09:38.478964: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 17:09:38.664606: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Prepararations for Model

In [3]:
# Model Settings from Hugging Face: Path, Length, Output Directory
TRAINING_MODEL_PATH = "microsoft/deberta-v3-base"
TRAINING_MAX_LENGTH = 1024
OUTPUT_DIR = "output"

In [4]:
train = pd.read_csv(open('/kaggle/input/data-exploration/deberta_train.csv'))
test = pd.read_csv(open('/kaggle/input/data-exploration/deberta_test.csv'))
train = train.drop(['full_text', 'tokens', 'trailing_whitespace', 'provided_labels', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-URL_PERSONAL', 'I-NAME_STUDENT', 'B-EMAIL'], axis = 1)
test = test.drop(['full_text', 'tokens', 'trailing_whitespace', 'provided_labels', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-URL_PERSONAL', 'I-NAME_STUDENT', 'B-EMAIL'], axis = 1)
print(train.columns)

Index(['document', 'input_ids', 'attention_mask', 'offset_mapping', 'labels'], dtype='object')


In [5]:
train['input_ids'] = train['input_ids'].apply(literal_eval)
train['attention_mask'] = train['attention_mask'].apply(literal_eval)
train['offset_mapping'] = train['offset_mapping'].apply(literal_eval)
train['labels'] = train['labels'].apply(literal_eval)
test['input_ids'] = test['input_ids'].apply(literal_eval)
test['attention_mask'] = test['attention_mask'].apply(literal_eval)
test['offset_mapping'] = test['offset_mapping'].apply(literal_eval)
test['labels'] = test['labels'].apply(literal_eval)

In [6]:
# DOWNSAMPLING: COMMENT ALL IF NOT DOWNSAMPLING

file = open("/kaggle/input/data-exploration/docs2cut.txt","r")
downsample_docs = [int(i) for i in file.read().split('\n')[:-1]]
print("Number of docs removed: ", len(downsample_docs))

print("Length of training data: ", len(train))
print("Length of testing data: ", len(test))

for d in downsample_docs: 
    train = train[train.document != d]
    test = test[test.document != d]
    
print("After downsampling length of training data: ", len(train))
print("After downsampling length of testing data: ", len(test))

Number of docs removed:  2931
Length of training data:  5695
Length of testing data:  1889
After downsampling length of training data:  3794
After downsampling length of testing data:  1263


In [7]:
ds_train = Dataset.from_pandas(train)
ds_test = Dataset.from_pandas(test)

# For final model
all_data = pd.concat([train, test], ignore_index=True, sort=False)
ds = Dataset.from_pandas(all_data)

In [8]:
# Obtain all possible labels, along with a map between them and their ids
LABELS = ['B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 
          'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM', 'I-NAME_STUDENT', 'I-PHONE_NUM', 
          'I-STREET_ADDRESS', 'I-URL_PERSONAL', 'O']

label2id = {l:i for i, l in enumerate(LABELS)}
id2label = {i:l for i, l in enumerate(LABELS)}
print(LABELS)

['B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM', 'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'I-URL_PERSONAL', 'O']


# Metrics


In [9]:
# METRICS

# Investigate adding some metrics to compute finer accuracies

# now let's define the metrics we care about 
from seqeval.metrics import recall_score, precision_score

def compute_metrics(p, all_labels):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [all_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [all_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    recall = recall_score(true_labels, true_predictions)
    precision = precision_score(true_labels, true_predictions)
    f5_score = (1 + 5*5) * recall * precision / (5*5*precision + recall)
    
    results = {
        'recall': recall,
        'precision': precision,
        'f5': f5_score
    }
    return results

# Custom Loss


In [10]:
# Implement custom loss function 
import torch.nn.functional as F

penalty = .05 # raise this to penalize guesses of PII for non-PII labels more
wts = torch.ones(len(LABELS))
wts[-1] = penalty # weight last class
wts = wts.to(device='cuda')
print('weights:', wts)

def custom_loss(preds, trues, epsilon=1):
    """
    cross-entropy loss, but if a token is NOT PII, the loss for the 
    corresponding prediction, instead of being -logsoftmax(logit), 
    is equal to epsilon. 
    
    This function will be applied to (logits, labels), i.e. we will
    call custom_loss(logits, labels) in the training step; logits will 
    have shape bs x max_seq_len x num_labels, labels will have shape 
    bs x max_seq_len. 
    
    It seems we can use just the built-in 'weight' parameter passed
    to the cross_entropy function
    """
    loss = F.cross_entropy(preds.permute(0,2,1), trues, 
                           weight=wts)
    
    return loss

# subclass Trainer 
class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def compute_loss(self, model, inputs, return_outputs=False):
        # inputs is a dict with keys 'input_ids', a tensor representing the 
        # input sequence as a str of ints which are the vocabulary indices; 
        # token_type_ids and attention_mask, which seem irrelevant for the 
        # compute_loss function; and 'labels' which is the true values       
        labels = inputs.pop('labels') # removes labels from inputs and assigns to new variable
        outputs = model(**inputs)
        logits = outputs[0]
        loss = custom_loss(logits, labels)
        
        return (loss, outputs) if return_outputs else loss

weights: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 0.0500], device='cuda:0')


# Trainer Class

In [11]:
# COLLATE: For batches
tokenizer = AutoTokenizer.from_pretrained(TRAINING_MODEL_PATH)
collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)

# training args, args setup
lr, epochs = 2e-5, 3
bs, wd = 4, 0.01

# model import and training 
# define model
model = AutoModelForTokenClassification.from_pretrained(
    TRAINING_MODEL_PATH,
    num_labels=len(LABELS),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

class CustomCallback(TrainerCallback):
    
    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer
    
    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = copy.deepcopy(control)
            self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
            return control_copy

args = TrainingArguments(
    output_dir=OUTPUT_DIR, 
    fp16=True,
    learning_rate=lr,
    num_train_epochs=epochs,
    per_device_train_batch_size=bs,
    gradient_accumulation_steps=2,
    report_to="none",
    evaluation_strategy="no",
    logging_strategy="epoch",
    do_eval=False,
    save_total_limit=1,
    logging_steps=20,
    lr_scheduler_type='cosine',
    metric_for_best_model="f5",
    greater_is_better=True,
    warmup_ratio=0.1,
    weight_decay=wd
)

trainer = CustomTrainer(
    model=model, 
    args=args, 
    train_dataset = ds,
    data_collator=collator, 
    tokenizer=tokenizer,
    compute_metrics=partial(compute_metrics, all_labels=LABELS),
)

trainer.add_callback(CustomCallback(trainer)) 

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `

In [12]:
train_result = trainer.train()

Step,Training Loss
316,0.345400
633,0.019200
948,0.012700


In [13]:
trainer.state.log_history

[{'loss': 0.3454,
  'grad_norm': 260794.1875,
  'learning_rate': 1.6866384115072917e-05,
  'epoch': 1.0,
  'step': 316},
 {'loss': 0.0192,
  'grad_norm': 3.3650460243225098,
  'learning_rate': 6.007912624571963e-06,
  'epoch': 2.0,
  'step': 633},
 {'loss': 0.0127,
  'grad_norm': 44931.6796875,
  'learning_rate': 0.0,
  'epoch': 3.0,
  'step': 948},
 {'train_runtime': 2852.381,
  'train_samples_per_second': 5.319,
  'train_steps_per_second': 0.332,
  'total_flos': 7548615217992288.0,
  'train_loss': 0.12575222470086336,
  'epoch': 3.0,
  'step': 948}]

In [14]:
#trainer.save_model("roberta_partition_downsample")
#tokenizer.save_pretrained("roberta_partition_downsample")

trainer.save_model("deberta_partition")
tokenizer.save_pretrained("deberta_partition")

('deberta_partition/tokenizer_config.json',
 'deberta_partition/special_tokens_map.json',
 'deberta_partition/spm.model',
 'deberta_partition/added_tokens.json',
 'deberta_partition/tokenizer.json')

In [15]:
#df_log_history = pd.DataFrame(trainer.state.log_history)
#df_log_history.to_csv("downsamplepenalty.07epoch6.csv", index=False)